In [59]:
import pandas as pd 
import numpy as np
import datetime
import time
# import matplotlib.pyplot as plt

# cant run outside docker tensorflow
from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, Activation
# from tensorflow.keras.layers import LSTM

In [2]:
dfday = pd.read_pickle('../../data/time_ecom/dfday.pkl')

In [26]:
def simple_splitter(df, test_len=28):
    """splits off the tail of the dataframe for timeseries split
    returns 2 dataframes"""
    df_train = df[:-test_len]
    df_test = df[-test_len:]
    return df_train, df_test

In [29]:
def scale_df2array(df):
    """input dataframe formatted for FBprophet
    columns = ['ds', 'y']
    returns array.shape=(len(df), 1) and the fit scaler for inverse transform
    """
    a_ray = np.array(df['y'].values).reshape(-1,1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_array = scaler.fit_transform(a_ray)
    return scaled_array, scaler

In [31]:
fullarray, thescaler = scale_df2array(dfday)
fullarray[:8]

array([[0.21338912],
       [0.56066946],
       [0.76987448],
       [0.83682008],
       [0.72384937],
       [0.24267782],
       [0.        ],
       [0.10460251]])

In [32]:
fullarray[fullarray > .99]

array([1.       , 0.9916318])

In [55]:
rnn_seq_len = 14
# train, test = simple_splitter(fullarray, test_len=28)
train, test = simple_splitter(fullarray, test_len=28 + 1 + rnn_seq_len)
train.shape, "train   :   test", test.shape

((94, 1), 'train   :   test', (43, 1))

In [23]:
np.random.seed(9)
def array2rnnformat(scaled_array, seq_len=14, do_shuffle=True):
    """if dfday then len(df)=137
    default len(df_test)=28 so len(df_train)=109
    X is sequece of arrays of len seq_len
    y is the very next singular future value after array X
    x=len(df), y=seq_len, z=num features (parallel timeseries)
    output = arrayshape(x,y,z)  keras LSTM wants this
    """
    seq_len += 1
    lstm_array = []  #  array of arrays len=seq_len
    for index in range(scaled_array.shape[0] - seq_len):
        lstm_array.append(scaled_array[index : index + seq_len])
    # convert the list to array
    lstm_array = np.array(lstm_array)
    # shuffle the array of arrays
    if do_shuffle:
        np.random.shuffle(lstm_array)
    # take the last 'column' value, this becomes y
    X_lstm = lstm_array[:, :-1]
    y_lstm = lstm_array[:, -1]
    # reshape X to (x,y,z) for LSTM input
    X_lstm = np.reshape(X_lstm, (X_lstm.shape[0], X_lstm.shape[1], 1))
    return X_lstm, y_lstm

In [56]:
X_train, y_train = array2rnnformat(train, seq_len=14, do_shuffle=True)

In [57]:
X_train[:3]
X_train.shape, "Xshape  :  yshape", y_train.shape

((79, 14, 1), 'Xshape  :  yshape', (79, 1))

In [58]:
X_test, Y_test = array2rnnformat(test, seq_len=rnn_seq_len, do_shuffle=False)
X_test.shape, "Xshape  :  yshape", y_test.shape

((28, 14, 1), 'Xshape  :  yshape', (28, 1))